# TUTORIAL GO
Presentado por:
- Andrés Felipe Ortega Fernández
- Camilo Andrés Chaparro Silva
- Christian Camilo Rojas Tapias
- Jhon Fredy Acosta Murillo

### EJERCICIO SIMPLE

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
)

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo.txt")
    if err != nil {
        panic(err)
	}

    final := 16777215
    for i := 0; i <= final; i++ {
        _, err = f.WriteString(fmt.Sprintf("%06x\n", i))
        if err != nil {
            panic(err)
        }
    }

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}



### EJERCICIO CON CONCURRENCIA

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
)

func calcNums(start, end int, f *os.File, doneCh chan struct{}){
	for i := start; i <= end; i++ {
		_, err := f.WriteString(fmt.Sprintf("%06x\n", i))
		if err != nil {
			panic(err)
		}
	}
	doneCh <- struct{}{}
}

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo_concurrente.txt")
    if err != nil {
        panic(err)
	}

	numGoRoutines := 10
	doneCh := make(chan struct{})

    final := 100000
    for i := 0; i <= final; i = i + (final/numGoRoutines) + 1 {
		paso := i + (final/numGoRoutines)
		if paso > final {
			paso = final
		}
		fmt.Printf("ejecutando %d %d\n", i, paso)
		go calcNums(i, paso, f, doneCh)
    }

	doneNum := 0
	for doneNum < numGoRoutines {
		<- doneCh
		fmt.Println("Terminé")
		doneNum++
	}
	fmt.Println("Todos han finalizado")

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}



### EJERCICIO CON CONCURRENCIA MEJORADO

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
	"strings"
)

func calcNums(start, end int, resultCh chan string, doneCh chan struct{}){
	var sBuilder strings.Builder
	for i := start; i <= end; i++ {
		fmt.Fprint(&sBuilder, fmt.Sprintf("%06x\n", i))
	}
	resultCh <- sBuilder.String()
	doneCh <- struct{}{}
}

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo_concurrente.txt")
    if err != nil {
        panic(err)
	}

	outCh := make(chan string)
	doneWrite := make(chan struct{})
	//Writer
	go func() {
		for s := range outCh {
			_, err := f.WriteString(s)
			if err != nil {
				panic(err)
			}
		}
		doneWrite <- struct{}{}
	}()

	numGoRoutines := 10
	doneCh := make(chan struct{})

    final := 100000
    for i := 0; i <= final; i = i + (final/numGoRoutines) + 1 {
		paso := i + (final/numGoRoutines)
		if paso > final {
			paso = final
		}
		fmt.Printf("ejecutando %d %d\n", i, paso)
		go calcNums(i, paso, outCh, doneCh)
    }

	doneNum := 0
	for doneNum < numGoRoutines {
		<- doneCh
		fmt.Println("Terminé")
		doneNum++
	}
	close(outCh)
	<-doneWrite
	fmt.Println("Todos han finalizado")

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}

